In [2]:
# ================================
# LoRA fine-tuning (Korean SFT)
# ================================
!pip -q install -U transformers peft accelerate torch

import os, random, re, json
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, set_seed
from peft import LoraConfig, get_peft_model, TaskType

# (중복 방어) W&B 비활성화
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"

# 1) 기본 설정
SEED = 42
set_seed(SEED)
rng = random.Random(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# 2) 모델/토크나이저
# ── 한국어 품질 권장: KoGPT2
BASE_MODEL = "skt/kogpt2-base-v2"
# 필요 시 distilgpt2로 바꾸실 수 있어요(한국어 성능은 낮을 수 있음):
# BASE_MODEL = "distilgpt2"

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
    tok.pad_token_id = tok.eos_token_id

# 템플릿 태그(토큰 경계 안정)
INST_TAG = "### 지시문:"
RESP_TAG = "### 응답:"
tok.add_special_tokens({"additional_special_tokens":[INST_TAG, RESP_TAG]})

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL)
model.resize_token_embeddings(len(tok))
model.to(device)

# 3) LoRA 설정 (GPT-2 계열 핵심 선형층만)
#   - c_attn : Q/K/V 합성 선형층
#   - c_proj : 어텐션/MLP 출력 선형층
#   - c_fc   : MLP 확장 선형층(mlp.c_fc를 넓게 매칭)
lora_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8, lora_alpha=16, lora_dropout=0.05,
    target_modules=["c_attn","c_proj","c_fc"]
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

# 4) 기본 한국어 지시문/응답 페어 (작지만 폭넓게)
pairs = [
    # 요약/한줄요약
    ("한 줄 요약: 연준 발표 이후 주식시장이 상승했다.", "연준 발표 이후 주가가 상승했다."),
    ("한 줄 요약(이모지 금지, 25자 이내): 비가 오고 교통 혼잡이 심해졌다.", "비로 인해 교통 혼잡이 심해졌다."),

    # 형식/톤 변환
    ("메일 첫 문장(공손체): 회의 일정 조율을 정중히 요청", "안녕하세요. 회의 일정 관련하여 가능한 시간을 여쭙고자 연락드립니다."),
    ("문장을 더 공손하게: 내일까지 자료 주세요.", "번거로우시겠지만 내일까지 자료를 공유해 주실 수 있을까요?"),
    ("반말로 바꾸기: 오늘 일정 확인 부탁드립니다.", "오늘 일정 확인해줘."),
    # 키워드/추출
    ("키워드 3개 추출: 인공지능이 의료 영상 판독을 보조해 정확도를 높였다.", "인공지능, 의료 영상, 정확도"),
    ("해시태그 3개 생성: 여름철 수분 섭취의 중요성", "#여름건강 #수분섭취 #열사병예방"),
    # 구조화(JSON)
    ("JSON으로 요약(키: title, sentiment): '서비스 개선 공지에 고객 반응이 대체로 긍정적이다.'",
     '{"title":"서비스 개선 공지 반응","sentiment":"positive"}'),
    ("항목별 요약(JSON: pros, cons): '배터리 오래가나 무게가 조금 무겁다.'",
     '{"pros":["배터리 오래감"],"cons":["무게가 다소 무거움"]}'),
    # 규칙/제약
    ("규칙: 한 문장, 마침표로 끝내기 — 딥러닝을 정의해줘.", "딥러닝은 다층 신경망으로 복잡한 패턴을 학습하는 기계학습 기법이다."),
    ("불릿 3개로 정리: 집중력을 높이는 방법", "• 방해 요소 제거\n• 짧은 목표 설정\n• 일정한 휴식"),
    # 설명/정의
    ("한 문장 정의: 머신러닝.", "머신러닝은 데이터에서 패턴을 학습해 예측을 수행하는 기술이다."),
    ("초보자에게 설명: 과적합이 뭐야?", "과적합은 학습 데이터에만 지나치게 맞춰져 새로운 데이터에서 성능이 떨어지는 현상이다."),
    # 분류/판정(간단)
    ("감정 분류(긍/부정 중 하나): '이 제품 정말 마음에 든다.'", "긍정"),
    ("감정 분류(긍/부정 중 하나): '배송이 너무 느려서 실망했다.'", "부정"),
    # 변환/요청
    ("문장을 더 간결하게: 본 건과 관련하여 검토 결과를 전달드립니다.", "관련 검토 결과를 전달드립니다."),
    ("숫자만 추출: 주문번호 A-001-39", "00139"),
    # 일정/비즈니스
    ("회의 아젠다 3개 제안(한 줄식): 온라인 세미나 준비 회의", "목표 확인\n역할 분담\n타임라인 확정"),
    ("납기 연장 요청 메일 첫 문장(공손체, 한 문장):", "안녕하세요, 납기 일정 관련하여 부득이하게 연장을 요청드리고자 연락드립니다."),
    # 스타일/재구성
    ("문장을 더 명확하게: 데이터 처리 속도가 부족하다.", "데이터 처리 속도가 느려 성능 저하가 발생한다."),
    ("두 문장을 한 문장으로: 모델은 정확하지만 느리다. 배치 크기를 줄였다.", "정확하지만 느린 모델을 개선하기 위해 배치 크기를 줄였다."),
]

# ---------------------------
# 4-1) 자동 증강 모듈
# ---------------------------
# 안전을 위해 "응답이 그대로 유효"한 형태의 지시문 변형만 수행합니다.
SYNSETS = [
    # 요약 계열
    (r"한 줄 요약", ["한 문장 요약", "한 줄로 요약", "요약(한 문장)"]),
    (r"\(이모지 금지, 25자 이내\)", ["(이모지 없이, 25자 이하)", "(25자 이내, 이모지 금지)"]),

    # 톤/형식
    (r"메일 첫 문장\(공손체\)", ["정중한 메일 시작 문장", "공손한 메일 첫 문장", "공손체 메일 서두"]),
    (r"문장을 더 공손하게", ["문장을 정중하게 표현", "문장을 공손체로 바꾸기", "더 공손한 표현으로"]),
    (r"반말로 바꾸기", ["반말로 변경", "반말체로 바꾸기", "반말 표현으로"]),
    # 추출/구조화
    (r"키워드 3개 추출", ["핵심어 3개 뽑기", "키워드 세 가지", "핵심 키워드 3개"]),
    (r"해시태그 3개 생성", ["해시태그 세 개 생성", "해시태그 3개 추천"]),
    (r"JSON으로 요약", ["JSON 형식으로 요약", "요약(JSON 포맷)"]),
    (r"항목별 요약", ["항목별 정리", "카테고리별 요약"]),
    # 규칙/제약
    (r"불릿 3개로 정리", ["불릿 세 개로 정리", "세 줄 불릿으로 정리"]),
    (r"규칙: 한 문장, 마침표로 끝내기", ["규칙: 한 문장으로, 마침표 필수", "규칙: 한 문장·마침표"]),
    # 정의/설명
    (r"한 문장 정의", ["한 줄 정의", "한 문장으로 정의"]),
    (r"초보자에게 설명", ["처음 배우는 사람에게 설명", "입문자에게 설명"]),
    # 분류
    (r"감정 분류\(긍/부정 중 하나\)", ["감정 판정(긍/부정)", "감정 분류(긍정/부정)"]),
    # 변환
    (r"문장을 더 간결하게", ["문장을 간결하게", "간결한 문장으로 바꾸기"]),
    (r"숫자만 추출", ["숫자만 남기기", "숫자만 출력하기"]),
    # 일정/비즈니스
    (r"회의 아젠다 3개 제안\(한 줄식\)", ["회의 아젠다 3가지(한 줄씩)", "회의 주제 3개(한 줄씩)"]),
    (r"납기 연장 요청 메일 첫 문장\(공손체, 한 문장\)", ["납기 연장 요청(공손체, 한 문장)", "납기 일정 연장 요청(공손체)"]),
    # 스타일
    (r"문장을 더 명확하게", ["문장을 명확하게", "더 명확한 문장으로"]),
    (r"두 문장을 한 문장으로", ["두 문장을 합쳐 한 문장으로", "두 문장을 하나로"]),
]

EMOJI_PATTERN = re.compile(
    "[\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"    # symbols & pictographs
    "\U0001F680-\U0001F6FF"    # transport & map
    "\U0001F1E0-\U0001F1FF"    # flags (iOS)
    "]+", flags=re.UNICODE
)

def has_emoji(s: str) -> bool:
    return bool(EMOJI_PATTERN.search(s))

def is_valid_json(s: str) -> bool:
    try:
        json.loads(s)
        return True
    except Exception:
        return False

def bullets_count(s: str) -> int:
    lines = [ln.strip() for ln in s.splitlines() if ln.strip()]
    # •, -, * 를 불릿으로 인정
    return sum(1 for ln in lines if ln.startswith("•") or ln.startswith("-") or ln.startswith("*"))

def digits_only(s: str) -> bool:
    return bool(re.fullmatch(r"[0-9]+", s.strip()))

def ends_with_period_one_sentence(s: str) -> bool:
    # 매우 단순한 검증: 마침표로 끝나고 문장부호가 여러 번 안 나오는지
    return s.strip().endswith(".")

def enforce_constraints_if_any(instr: str, resp: str) -> bool:
    """지시문에 담긴 제약 조건을 응답이 만족하는지 간단히 검증."""
    # 이모지 금지
    if "이모지" in instr and has_emoji(resp):
        return False
    # 글자 수 제한: '25자' / '25자 이내' 등 → 숫자 추출
    m = re.search(r"(\d+)\s*자", instr)
    if m:
        limit = int(m.group(1))
        if len(resp) > limit:
            return False
    # 불릿 3개
    if ("불릿 3" in instr or "불릿 세" in instr) and bullets_count(resp) != 3:
        return False
    # JSON
    if "JSON" in instr or "Json" in instr or "json" in instr:
        if not is_valid_json(resp):
            return False
    # 숫자만
    if "숫자만" in instr:
        if not digits_only(resp):
            return False
    # 한 문장 + 마침표
    if ("한 문장" in instr or "한 문장으로" in instr) and ("마침표" in instr or "마침표로" in instr):
        if not ends_with_period_one_sentence(resp):
            return False
    return True

def paraphrase_one(instr: str) -> str:
    """사전 정의된 동의어 패턴 중 일부를 랜덤 적용하여 지시문을 변형."""
    out = instr
    # 0~3개 랜덤 적용
    n_apply = rng.randint(0, 3)
    cand = rng.sample(SYNSETS, k=min(n_apply, len(SYNSETS)))
    for pat, repls in cand:
        if rng.random() < 0.7:  # 적용 확률
            out = re.sub(pat, rng.choice(repls), out)
    # 가벼운 접두/접미 변형
    add_prefix = rng.choice(["", "요청: ", "지시: ", "규칙: "])
    add_suffix = rng.choice(["", "", "", " (간단히)", " (한 줄)", " (명확하게)"])
    out = f"{add_prefix}{out}{add_suffix}".strip()
    return out

def augment_pairs(pairs, per_src=3, max_trials=10):
    """각 (instr, resp)에 대해 의미를 해치지 않는 선에서 지시문을 per_src개 생성."""
    aug = []
    for instr, resp in pairs:
        seen = set([instr])
        aug.append((instr, resp))  # 원본 포함
        trials = 0
        made = 0
        while made < per_src and trials < max_trials:
            trials += 1
            cand = paraphrase_one(instr)
            if cand in seen:
                continue
            # 응답은 동일하게 유지 가능한 변형만 사용 → 제약 검증 통과해야 채택
            if enforce_constraints_if_any(cand, resp):
                aug.append((cand, resp))
                seen.add(cand)
                made += 1
    return aug

# 증강 실행
PER_SRC = 3  # 각 원본 지시문당 증강 개수(원본+3 = 4배)
aug_pairs = augment_pairs(pairs, per_src=PER_SRC)

print(f"[증강] 원본 {len(pairs)} → 증강 후 {len(aug_pairs)} 샘플")
print("샘플 5개 미리보기:")
for i, (ins, out) in enumerate(aug_pairs[:5]):
    print(f"- {ins}  ///  {out}")

# 템플릿 결합 함수
def ex(prompt, answer):
    return f"{INST_TAG}\n{prompt}\n\n{RESP_TAG}\n{answer}{tok.eos_token}"

# 텍스트화
texts = [ex(p, a) for p, a in aug_pairs]

# 데이터가 충분히 늘었으므로 REPEAT는 낮게 유지
REPEAT = 10
texts = texts * REPEAT

# 5) 커스텀 데이터셋 (토큰화는 collator에서 배치로 처리)
class TextOnlyDS(Dataset):
    def __init__(self, texts): self.texts = texts
    def __len__(self): return len(self.texts)
    def __getitem__(self, i): return {"text": self.texts[i]}

split = int(len(texts)*0.9)
train_ds = TextOnlyDS(texts[:split])
val_ds   = TextOnlyDS(texts[split:])

# 6) Collator: 배치 토큰화 + "응답만 로스" 마스킹 + 안전 패딩
RESP_TAG_WITH_NL = RESP_TAG + "\n"
resp_ids = tok(RESP_TAG_WITH_NL, add_special_tokens=False)["input_ids"]

def find_subseq(seq, sub):
    L, l = len(seq), len(sub)
    for i in range(L - l + 1):
        if seq[i:i+l] == sub:
            return i
    return -1

def response_only_collate(features, tokenizer=tok, max_length=384):
    # 텍스트 배치 토큰화
    texts = [f["text"] for f in features]
    enc = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    input_ids = enc["input_ids"]
    attn = enc["attention_mask"]

    # labels 초기화(-100), "응답:" 이후만 정답 복사
    labels = torch.full_like(input_ids, -100)
    B = input_ids.size(0)
    for i in range(B):
        ids = input_ids[i].tolist()
        pos = find_subseq(ids, resp_ids)
        if pos != -1:
            start = pos + len(resp_ids)
            seq_len = int(attn[i].sum().item())  # pad 제외
            start = min(start, seq_len)
            labels[i, start:seq_len] = input_ids[i, start:seq_len]

    return {"input_ids": input_ids, "attention_mask": attn, "labels": labels}

def collate(batch):
    return response_only_collate(batch)

# 7) 학습 인자 (버전 호환: 최신/구버전 자동 fallback)
model.config.use_cache = False  # Trainer 경고 방지
def make_args():
    try:
        return TrainingArguments(
            output_dir="./out_lora",
            overwrite_output_dir=True,
            num_train_epochs=4,
            per_device_train_batch_size=4,
            gradient_accumulation_steps=2,
            learning_rate=1e-4,
            lr_scheduler_type="cosine",
            weight_decay=0.0,
            logging_steps=50,
            evaluation_strategy="steps",
            eval_steps=200,
            save_strategy="no",
            remove_unused_columns=False,
            report_to="none",
            bf16=(torch.cuda.is_available() and getattr(torch.cuda, "is_bf16_supported", lambda: False)()),
            fp16=(torch.cuda.is_available() and not getattr(torch.cuda, "is_bf16_supported", lambda: False)()),
        )
    except TypeError:
        return TrainingArguments(
            output_dir="./out_lora",
            overwrite_output_dir=True,
            num_train_epochs=4,
            per_device_train_batch_size=4,
            gradient_accumulation_steps=2,
            learning_rate=1e-4,
            logging_steps=50,
            remove_unused_columns=False,
        )

args = make_args()

trainer = Trainer(
    model=model, args=args,
    train_dataset=train_ds, eval_dataset=val_ds,
    data_collator=collate, tokenizer=tok
)

print("학습 시작...")
trainer.train()
print("학습 종료.")

# 8) 어댑터 저장
save_dir = "./lora_adapter_ko_sft_aug"
os.makedirs(save_dir, exist_ok=True)
model.save_pretrained(save_dir)
tok.save_pretrained(save_dir)
print("저장:", save_dir)

# 9) 추론 함수(학습 템플릿과 동일하게!)
@torch.inference_mode()
def generate_ko(prompt, max_new_tokens=80, temperature=0.7, top_p=0.9, do_sample=False):
    model.eval()
    prefix = f"{INST_TAG}\n{prompt}\n\n{RESP_TAG}\n"
    inputs = tok(prefix, return_tensors="pt").to(model.device)
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample, temperature=temperature, top_p=top_p,
        pad_token_id=tok.eos_token_id, eos_token_id=tok.eos_token_id
    )
    txt = tok.decode(out[0], skip_special_tokens=False)
    return txt.split(RESP_TAG, 1)[-1].strip() if RESP_TAG in txt else txt.strip()

print("\n=== 데모 ===")
tests = [
    # 학습 분포와 유사(일반화 체크)
    "한 줄 요약: 배터리 수명이 길어 사용자 만족도가 높아졌다.",
    "정중한 메일 시작 문장: 일정 재조율 요청",
    "핵심 키워드 3개: 신규 기능 출시로 사용자 유입이 증가했다.",
    "JSON 형식으로 요약(키: title, sentiment): '배송 지연 이슈가 사라져 평점이 올랐다.'",
    # 새 변형(증강 전이 성능 체크)
    "요약(한 문장): 서버 과부하로 장애 발생, 임시 확장 적용.",
    "불릿 세 개로 정리: 업무 집중력을 높이는 실천 팁",
    "문장을 공손체로 바꾸기: 회의 일정 조율 부탁드립니다.",
    "한 줄 정의: 과소적합.",
    "감정 판정(긍/부정): '포장이 엉망이라 실망스러웠다.'",
]
for p in tests:
    print("Q:", p)
    print("A:", generate_ko(p))
    print("-"*40)


Device: cuda


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-4060551584.py:326: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 51200, 'bos_token_id': 51200, 'pad_token_id': 51200}.


trainable params: 1,179,648 || all params: 126,345,984 || trainable%: 0.9337
[증강] 원본 21 → 증강 후 84 샘플
샘플 5개 미리보기:
- 한 줄 요약: 연준 발표 이후 주식시장이 상승했다.  ///  연준 발표 이후 주가가 상승했다.
- 한 줄 요약: 연준 발표 이후 주식시장이 상승했다. (명확하게)  ///  연준 발표 이후 주가가 상승했다.
- 규칙: 한 줄 요약: 연준 발표 이후 주식시장이 상승했다.  ///  연준 발표 이후 주가가 상승했다.
- 요청: 한 문장 요약: 연준 발표 이후 주식시장이 상승했다.  ///  연준 발표 이후 주가가 상승했다.
- 한 줄 요약(이모지 금지, 25자 이내): 비가 오고 교통 혼잡이 심해졌다.  ///  비로 인해 교통 혼잡이 심해졌다.
학습 시작...


Step,Training Loss
50,3.810000
100,2.016700
150,1.301100
200,1.012600
250,0.929400
300,0.962500
350,0.860500


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


학습 종료.


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


저장: ./lora_adapter_ko_sft_aug

=== 데모 ===
Q: 한 줄 요약: 배터리 수명이 길어 사용자 만족도가 높아졌다.
A: {"배터리 수명"}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}
----------------------------------------
Q: 정중한 메일 시작 문장: 일정 재조율 요청
A: 안녕하세요, 일정 관련하여 부득이하게 연장을 요청드리고자 연락드립니다.00139<unk>hanmail.com에서 확인할 수 있습니다.00139<unk>00139<unk>00139<unk>00139<unk>00139<unk>00139<unk>00139<unk>00139<unk>00139<unk>00139<unk>00139<unk>0013
----------------------------------------
Q: 핵심 키워드 3개: 신규 기능 출시로 사용자 유입이 증가했다.
A: • 신규 기능 출시로 사용자 유입이 증가했다.</d> #201809139 #미세먼지 #초미세먼지 #초미세먼지종말론 #초미세먼지종말론 #초미세먼지정체성 #초미세먼지정체성폭포틱한정도로 인해 성능 저
----------------------------------------
Q: JSON 형식으로 요약(키: title, sentiment): '배송 지연 이슈가 사라져 평점이 올랐다.'
A: {"title":"배송 지연 이슈가 사라져 평점이 상승했다.}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}}
----------------------------------------
Q: 요약(한 문장): 서버 과부하로 장애 발생, 임시 확장 적용.
A: {"positive":"positive":"positive"}¶{"positive"}¶{"positive"}¶{"positive"}¶{"positive"}¶{"positive"